# 1. Installing Dependencies

!pip install -q datasets transformers accelerate timm
!pip install -q -U albumentations>=1.4.5 torchmetrics pycocotools

In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim import Adam, SGD
import torch.nn.functional as F

import torchvision
from torchvision.transforms import transforms
from torchvision.models import models